This notebook is used to repair data to remove the effect of sensitive attributes using quantiles to retrieve new(generated) data.

In [33]:
# !pip install numpy
# !pip install tensorflow


In [34]:
import numpy as np
import random
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dense, BatchNormalization, Dropout
from tensorflow.keras import optimizers
from sklearn.metrics import precision_score,recall_score, accuracy_score,confusion_matrix,f1_score
from tensorflow import feature_column
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import History

import pandas as pd



# Data Repairing starts here


In [35]:

file_out = pd.read_csv('Data/adult_processed.csv')
cols = []
for i in list(file_out.columns):
    if  i != 'income':
        cols.append(i)

feature_set1 = pd.read_csv('Data/train.csv')
feature_set2 = pd.read_csv('Data/test.csv')

print(feature_set1['fnlwgt'])
x = feature_set1[cols].copy().values
y = feature_set1[['income']].copy().values
        
sc = StandardScaler()
X_train = sc.fit_transform(x)
y_train = y

x2 = feature_set2[cols].copy().values
y2 = feature_set2[['income']].copy().values
        
X_test = sc.transform(x2)
y_test = y2


X_test.shape, y_test.shape 

0        158847
1        709445
2        225970
3        155233
4        235894
          ...  
39068    452353
39069    390657
39070    178922
39071    151817
39072    121468
Name: fnlwgt, Length: 39073, dtype: int64


((9768, 94), (9768, 1))

# Gender

In [36]:

female_features =  feature_set1[feature_set1['gender_Female']==1]
x_female = female_features[cols].copy().values
y_female = female_features[['income']].copy().values

X_train_female = sc.fit_transform(x_female)
y_train_female = y_female

male_features = feature_set1[feature_set1['gender_Male']==1] 
x_male = male_features[cols].copy().values
y_male = male_features[['income']].copy().values

X_train_male = sc.fit_transform(x_male)
y_train_male = y_male



# checks
X_train.shape, X_train_female.shape, X_train_male.shape


((39073, 94), (12987, 94), (26086, 94))

In [37]:
fnlwgt_female = female_features['fnlwgt'].copy().values
print(fnlwgt_female)

fnlwgt_male = male_features['fnlwgt'].copy().values
print(fnlwgt_male)

[158847 155233 380633 ... 452353 178922 151817]
[709445 225970 235894 ... 312055 390657 121468]


In [38]:
# fq1 = np.quantile(fnlwgt_female, .25)
# mq1 = np.quantile(fnlwgt_male, .25)
# fq2 = np.quantile(fnlwgt_female, .50)
# mq2 = np.quantile(fnlwgt_male, .50)
# fq3 = np.quantile(fnlwgt_female, .75)
# mq3 = np.quantile(fnlwgt_male, .75)
# fq4 = np.quantile(fnlwgt_female, 1)
# mq4 = np.quantile(fnlwgt_male, 1)

fq1 = np.quantile(fnlwgt_female, .1)
mq1 = np.quantile(fnlwgt_male, .1)
fq2 = np.quantile(fnlwgt_female, .2)
mq2 = np.quantile(fnlwgt_male, .2)
fq3 = np.quantile(fnlwgt_female, .3)
mq3 = np.quantile(fnlwgt_male, .3)
fq4 = np.quantile(fnlwgt_female, .4)
mq4 = np.quantile(fnlwgt_male, .4)
fq5 = np.quantile(fnlwgt_female, .5)
mq5 = np.quantile(fnlwgt_male, .5)
fq6 = np.quantile(fnlwgt_female, .6)
mq6 = np.quantile(fnlwgt_male, .6)
fq7 = np.quantile(fnlwgt_female, .7)
mq7 = np.quantile(fnlwgt_male, .7)
fq8 = np.quantile(fnlwgt_female, .8)
mq8 = np.quantile(fnlwgt_male, .8)
fq9 = np.quantile(fnlwgt_female, .9)
mq9 = np.quantile(fnlwgt_male, .9)
fq10 = np.quantile(fnlwgt_female, 1)
mq10 = np.quantile(fnlwgt_male, 1)


# print(fq1, mq1)
# print(fq2, mq2)
# print(fq3, mq3)
# print(fq4, mq4)

q1 = (fq1 + mq1)/2
q2 = (fq2 + mq2)/2
q3 = (fq3 + mq3)/2
q4 = (fq4 + mq4)/2
q5 = (fq5 + mq5)/2
q6 = (fq6 + mq6)/2
q7 = (fq7 + mq7)/2
q8 = (fq8 + mq8)/2
q9 = (fq9 + mq9)/2
q10 = (fq10 + mq10)/2

female_quantiles = [fq1, fq2,fq3, fq4, fq5, fq6, fq7, fq8, fq9, fq10]
male_quantiles = [mq1, mq2, mq3, mq4, mq5, mq6, mq7, mq8, mq9, mq10]
quantiles = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10]
print("total: ",quantiles, "\n")
print("female: ",female_quantiles, "\n")
print("male: ", male_quantiles, "\n")



total:  [65787.25, 105738.9, 130517.4, 157462.8, 178017.0, 195791.0, 219234.09999999998, 259116.5, 325122.69999999995, 1470070.0] 

female:  [66250.00000000001, 104439.8, 128505.79999999999, 155103.6, 175642.0, 193090.0, 213817.19999999998, 252426.00000000003, 317537.39999999997, 1484705] 

male:  [65324.5, 107038.0, 132529.0, 159822.0, 180392.0, 198492.0, 224651.0, 265807.0, 332708.0, 1455435] 



In [39]:
# female_features["fnlwgt"] = np.where(female_features["fnlwgt"]<= fq1, q1,female_features["fnlwgt"] )
female_features =  feature_set1[feature_set1['gender_Female']==1]
final_female  = pd.DataFrame()
for i in range(10): 
    female_fts = female_features[female_features["fnlwgt"]<= female_quantiles[i]]
    female_fts["fnlwgt"] = quantiles[i]
    
    female_features = female_features.drop(female_fts.index)
    final_female = pd.concat([final_female, female_fts])
    
    

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\3184539871.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  female_fts["fnlwgt"] = quantiles[i]


In [40]:
# female_features

In [41]:
# final_female

In [42]:
male_features = feature_set1[feature_set1['gender_Male']==1] 
final_male  = pd.DataFrame()
for i in range(10): 
    male_fts = male_features[male_features["fnlwgt"]<= male_quantiles[i]]
    male_fts["fnlwgt"] = quantiles[i]
    
    male_features = male_features.drop(male_fts.index)
    final_male = pd.concat([final_male, male_fts])

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\1457001410.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  male_fts["fnlwgt"] = quantiles[i]


In [43]:
# male_features

In [44]:
# final_male

In [45]:
  feature_set1

,Unnamed: 0,age,fnlwgt,education,educational-num,capital-gain,capital-loss,hours-per-week,income,marital-status_Divorced,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
0,36945,77,158847,8.0,11,0,0,25,0,False,...,False,False,False,False,False,False,False,True,False,False
1,37802,37,709445,11.0,9,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
2,43538,61,225970,11.0,9,0,0,40,1,False,...,False,False,False,False,False,False,False,True,False,False
3,104,52,155233,11.0,9,0,0,28,0,False,...,False,False,False,False,False,False,False,True,False,False
4,25439,23,235894,1.0,7,0,0,55,0,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39068,1539,38,452353,15.0,10,0,0,40,0,True,...,False,False,False,False,False,False,False,True,False,False
39069,48723,25,390657,15.0,10,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
39070,32005,45,178922,11.0,9,0,0,20,0,False,...,False,False,False,False,False,False,False,True,False,False
39071,14825,45,151817,12.0,14,0,0,36,0,False,...,False,False,False,False,False,False,False,True,False,False


In [46]:
repaired_data = pd.DataFrame()
repaired_data = pd.concat([final_male, final_female])
repaired_data

# drop sensitive columns

,Unnamed: 0,age,fnlwgt,education,educational-num,capital-gain,capital-loss,hours-per-week,income,marital-status_Divorced,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
14,6280,32,65787.25,11.0,9,0,0,40,0,True,...,False,False,False,False,False,False,False,True,False,False
24,9443,68,65787.25,12.0,14,0,0,24,0,False,...,False,False,False,False,False,False,False,True,False,False
54,9197,31,65787.25,15.0,10,0,0,50,0,True,...,False,False,False,False,False,False,False,True,False,False
74,14568,81,65787.25,15.0,10,0,2174,35,1,False,...,False,False,False,False,False,False,False,True,False,False
110,8767,68,65787.25,11.0,9,0,0,65,0,False,...,False,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38962,21491,24,1470070.00,15.0,10,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
38990,3699,30,1470070.00,0.0,6,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
39020,33363,24,1470070.00,11.0,9,0,0,45,0,False,...,False,False,False,False,False,False,False,False,False,False
39043,41775,23,1470070.00,7.0,12,0,0,25,0,False,...,False,False,False,False,False,False,False,True,False,False


In [47]:
repaired_data = shuffle(repaired_data)
repaired_data

,Unnamed: 0,age,fnlwgt,education,educational-num,capital-gain,capital-loss,hours-per-week,income,marital-status_Divorced,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
10411,41693,47,325122.70,11.0,9,0,0,40,0,True,...,False,False,False,False,False,False,False,True,False,False
18306,17528,49,157462.80,11.0,9,0,0,40,0,True,...,False,False,False,False,False,False,False,True,False,False
37558,16471,38,178017.00,15.0,10,0,0,28,0,True,...,False,False,False,False,False,False,False,True,False,False
19346,16532,42,1470070.00,9.0,13,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
15162,21111,52,195791.00,4.0,3,0,0,40,0,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15981,1989,27,259116.50,0.0,6,0,0,40,0,True,...,False,False,False,False,False,False,False,True,False,False
36081,43786,26,65787.25,4.0,3,0,0,40,0,False,...,False,False,False,False,False,False,False,False,False,False
30733,5044,23,65787.25,11.0,9,0,0,40,0,False,...,False,False,False,False,False,False,False,True,False,False
9919,33714,41,130517.40,15.0,10,2174,0,45,0,True,...,False,False,False,False,False,False,False,True,False,False


In [48]:
# repaired_data['gender_Female'] = 0
# repaired_data['gender_Male'] = 0

# Age

In [49]:
age_bar = 30

features_above_bar = repaired_data[repaired_data['age']>age_bar]
x_above_age = features_above_bar[cols].copy().values
y_above_age = features_above_bar[['income']].copy().values

X_train_above_age = sc.fit_transform(x_above_age)
y_train_above_age = y_above_age

features_below_bar = repaired_data[repaired_data['age']<=age_bar]
x_below_age = features_below_bar[cols].copy().values
y_below_age = features_below_bar[['income']].copy().values

X_train_below_age = sc.fit_transform(x_below_age)
y_train_below_age = y_below_age

# checks
X_train.shape, X_train_above_age.shape, X_train_below_age.shape

((39073, 94), (26432, 94), (12641, 94))

In [50]:
fnlwgt_above = features_above_bar['fnlwgt'].copy().values
print(fnlwgt_above)

fnlwgt_below = features_below_bar['fnlwgt'].copy().values
print(fnlwgt_below)

[325122.7 157462.8 178017.  ... 195791.  130517.4 219234.1]
[157462.8  325122.7  105738.9  ... 259116.5   65787.25  65787.25]


In [51]:
fq1 = np.quantile(fnlwgt_above, .1)
mq1 = np.quantile(fnlwgt_below, .1)
fq2 = np.quantile(fnlwgt_above, .2)
mq2 = np.quantile(fnlwgt_below, .2)
fq3 = np.quantile(fnlwgt_above, .3)
mq3 = np.quantile(fnlwgt_below, .3)
fq4 = np.quantile(fnlwgt_above, .4)
mq4 = np.quantile(fnlwgt_below, .4)
fq5 = np.quantile(fnlwgt_above, .5)
mq5 = np.quantile(fnlwgt_below, .5)
fq6 = np.quantile(fnlwgt_above, .6)
mq6 = np.quantile(fnlwgt_below, .6)
fq7 = np.quantile(fnlwgt_above, .7)
mq7 = np.quantile(fnlwgt_below, .7)
fq8 = np.quantile(fnlwgt_above, .8)
mq8 = np.quantile(fnlwgt_below, .8)
fq9 = np.quantile(fnlwgt_above, .9)
mq9 = np.quantile(fnlwgt_below, .9)
fq10 = np.quantile(fnlwgt_above, 1)
mq10 = np.quantile(fnlwgt_below, 1)

q1 = (fq1 + mq1)/2
q2 = (fq2 + mq2)/2
q3 = (fq3 + mq3)/2
q4 = (fq4 + mq4)/2
q5 = (fq5 + mq5)/2
q6 = (fq6 + mq6)/2
q7 = (fq7 + mq7)/2
q8 = (fq8 + mq8)/2
q9 = (fq9 + mq9)/2
q10 = (fq10 + mq10)/2

above_quantiles = [fq1, fq2,fq3, fq4, fq5, fq6, fq7, fq8, fq9, fq10]
below_quantiles = [mq1, mq2, mq3, mq4, mq5, mq6, mq7, mq8, mq9, mq10]
quantiles2 = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10]
print("total: ",quantiles2, "\n")
print("female: ",above_quantiles, "\n")
print("male: ", below_quantiles, "\n")


total:  [85763.075, 118128.15, 143990.09999999998, 167739.9, 186904.0, 207512.55, 239175.3, 292119.6, 897596.35, 1470070.0] 

female:  [65787.25, 105738.9, 130517.4, 157462.8, 178017.0, 195791.0, 219234.09999999998, 259116.5, 325122.69999999995, 1470070.0] 

male:  [105738.9, 130517.4, 157462.8, 178017.0, 195791.0, 219234.09999999998, 259116.5, 325122.69999999995, 1470070.0, 1470070.0] 



In [52]:
above_features =  repaired_data[repaired_data['age']>age_bar]
final_above  = pd.DataFrame()
for i in range(10): 
    above_fts = above_features[above_features["fnlwgt"]<= above_quantiles[i]]
    above_fts["fnlwgt"] = quantiles2[i]
    
    above_features = above_features.drop(above_fts.index)
#     print("\n")
#     print((above_features.shape), final_above.shape)
    final_above = pd.concat([final_above, above_fts])
#     print(above_fts["fnlwgt"], above_quantiles[i])

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\3875210117.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  above_fts["fnlwgt"] = quantiles2[i]


In [53]:
below_features = repaired_data[repaired_data['age']<=age_bar]
final_below  = pd.DataFrame()
for i in range(10): 
    below_fts = below_features[below_features["fnlwgt"]<= below_quantiles[i]]
    below_fts["fnlwgt"] = quantiles2[i]
    below_features = below_features.drop(below_fts.index)
    final_below = pd.concat([final_below, below_fts])

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\2896898300.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  below_fts["fnlwgt"] = quantiles2[i]


In [54]:
repaired_data = pd.concat([final_below, final_above])
repaired_data = shuffle(repaired_data)
# repaired_data['age'] = 0

# Race (whites and non-whites)

In [55]:
features_white = repaired_data[repaired_data['race_White']==1]
x_white = features_white[cols].copy().values
y_white = features_white[['income']].copy().values

X_train_white = sc.fit_transform(x_white)
y_train_white = y_white

features_non_white = repaired_data[repaired_data['race_White']==0]
x_non_white = features_non_white[cols].copy().values
y_non_white = features_non_white[['income']].copy().values

X_train_non_white = sc.fit_transform(x_non_white)
y_train_non_white = y_non_white

# checks
X_train.shape, X_train_white.shape, X_train_non_white.shape

((39073, 94), (33384, 94), (5689, 94))

In [56]:
fnlwgt_white = features_white['fnlwgt'].copy().values
print(fnlwgt_white)

fnlwgt_non_white = features_non_white['fnlwgt'].copy().values
print(fnlwgt_non_white)

[ 207512.55  118128.15  239175.3  ...  239175.3   239175.3  1470070.  ]
[239175.3  167739.9  167739.9  ... 897596.35 292119.6  897596.35]


In [57]:
fq1 = np.quantile(fnlwgt_white, .1)
mq1 = np.quantile(fnlwgt_non_white, .1)
fq2 = np.quantile(fnlwgt_white, .2)
mq2 = np.quantile(fnlwgt_non_white, .2)
fq3 = np.quantile(fnlwgt_white, .3)
mq3 = np.quantile(fnlwgt_non_white, .3)
fq4 = np.quantile(fnlwgt_white, .4)
mq4 = np.quantile(fnlwgt_non_white, .4)
fq5 = np.quantile(fnlwgt_white, .5)
mq5 = np.quantile(fnlwgt_non_white, .5)
fq6 = np.quantile(fnlwgt_white, .6)
mq6 = np.quantile(fnlwgt_non_white, .6)
fq7 = np.quantile(fnlwgt_white, .7)
mq7 = np.quantile(fnlwgt_non_white, .7)
fq8 = np.quantile(fnlwgt_white, .8)
mq8 = np.quantile(fnlwgt_non_white, .8)
fq9 = np.quantile(fnlwgt_white, .9)
mq9 = np.quantile(fnlwgt_non_white, .9)
fq10 = np.quantile(fnlwgt_white, 1)
mq10 = np.quantile(fnlwgt_non_white, 1)

q1 = (fq1 + mq1)/2
q2 = (fq2 + mq2)/2
q3 = (fq3 + mq3)/2
q4 = (fq4 + mq4)/2
q5 = (fq5 + mq5)/2
q6 = (fq6 + mq6)/2
q7 = (fq7 + mq7)/2
q8 = (fq8 + mq8)/2
q9 = (fq9 + mq9)/2
q10 = (fq10 + mq10)/2

white_quantiles = [fq1, fq2,fq3, fq4, fq5, fq6, fq7, fq8, fq9, fq10]
non_white_quantiles = [mq1, mq2, mq3, mq4, mq5, mq6, mq7, mq8, mq9, mq10]
quantiles3 = [q1, q2, q3, q4, q5, q6, q7, q8, q9, q10]
print("total: ",quantiles3, "\n")
print("white: ",white_quantiles, "\n")
print("non white: ", non_white_quantiles, "\n")

total:  [85763.075, 118128.15, 143990.09999999998, 167739.9, 197208.275, 223343.925, 265647.44999999995, 594857.975, 897596.35, 1470070.0] 

white:  [85763.075, 118128.15, 143990.09999999998, 167739.9, 186904.0, 207512.55, 239175.3, 292119.6, 897596.35, 1470070.0] 

non white:  [85763.075, 118128.15, 143990.09999999998, 167739.9, 207512.55, 239175.3, 292119.6, 897596.35, 897596.35, 1470070.0] 



In [58]:
white_features =  repaired_data[repaired_data['race_White']==1]
final_white  = pd.DataFrame()
for i in range(10): 
    white_fts = white_features[white_features["fnlwgt"]<= white_quantiles[i]]
    white_fts["fnlwgt"] = quantiles2[i]
    
    white_features = white_features.drop(white_fts.index)
#     print("\n")
#     print((above_features.shape), final_above.shape)
    final_white = pd.concat([final_white, white_fts])

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\1041614813.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  white_fts["fnlwgt"] = quantiles2[i]


In [59]:
non_white_features = repaired_data[repaired_data['race_White']==0] 
final_non_white  = pd.DataFrame()
for i in range(10): 
    non_white_fts = non_white_features[non_white_features["fnlwgt"]<= non_white_quantiles[i]]
    non_white_fts["fnlwgt"] = quantiles2[i]
    non_white_features = non_white_features.drop(non_white_fts.index)
    final_non_white = pd.concat([final_non_white, non_white_fts])

C:\Users\riash\AppData\Local\Temp\ipykernel_26216\1587552793.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_white_fts["fnlwgt"] = quantiles2[i]


In [60]:
repaired_data = pd.concat([final_non_white, final_white])
repaired_data = shuffle(repaired_data)
race_atrributes =  [ 
    'race_Amer-Indian-Eskimo',
    'race_Asian-Pac-Islander',
    'race_Black',
    'race_Other',
    'race_White']
# for i in race_atrributes:
# #     print(repaired_data[i] )
#     repaired_data[i] = 0

In [61]:
# similar procedures can be followed to repair data for other sensitive attributes. 
# Once all such biases are removed, the final dataframe "repaired_data" can be saved as a csv to the project folder. 

In [62]:
# sensitive_atrributes =  [ 'age',
#     'race_Amer-Indian-Eskimo',
#     'race_Asian-Pac-Islander',
#     'race_Black',
#     'race_Other',
#     'race_White',
#     'gender_Female',
#     'gender_Male']
# for i in sensitive_atrributes:
#     print(repaired_data[i] )
#     repaired_data[i] = 0

In [63]:
repaired_data = repaired_data.drop('Unnamed: 0', axis = 'columns')

In [64]:
repaired_data.to_csv('Data/train_data_repaired.csv') # saves new training dataframe in the given train file